In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 8, 5
plt.rcParams['font.size'] = 12
plt.rcParams['savefig.format'] = 'pdf'
sns.set_style('darkgrid')

In [2]:
import networkx as nx

G: nx.classes.digraph.DiGraph = nx.read_edgelist('data/yekaterinburg_small_speed_history.edgelist', create_using=nx.DiGraph())
len(G.edges()), len(G.nodes)

(5631, 3087)

In [3]:
import folium

m = folium.Map(location=[56.896876, 60.589692])

x_min = 60.6051
x_max = 60.6422
y_min = 56.8178
y_max = 56.8418

new_G = G.copy()

for u, v, data in G.edges(data=True):
    if (
        x_min <= data['x_coordinate_start'] <= x_max
        and x_min <= data['x_coordinate_end'] <= x_max
        and y_min <= data['y_coordinate_start'] <= y_max
        and y_min <= data['y_coordinate_end'] <= y_max
    ):
        line = folium.PolyLine(locations=[
            (data['y_coordinate_start'], data['x_coordinate_start']),
            (data['y_coordinate_end'], data['x_coordinate_end'])
        ], color='blue', weight=5, opacity=0.8, popup=f"Speed: {(data['y_coordinate_start'], data['x_coordinate_start']), (data['y_coordinate_end'], data['x_coordinate_end'])} km/h")
        line.add_to(m)
    else:
        new_G.remove_edge(u, v)

m

In [4]:
isolated_nodes = list(nx.isolates(new_G))
new_G.remove_nodes_from(isolated_nodes)
len(new_G.edges()), len(new_G.nodes)

(1235, 796)

In [5]:
from collections import defaultdict
unique_non_directed_edges = defaultdict(int)

for u, v, data in new_G.edges(data=True):
    assert u != v, 'There are loops'
    edge = (u, v) if u < v else (v, u)
    unique_non_directed_edges[edge] += 1

value_counts = defaultdict(int)
for value in unique_non_directed_edges.values():
    value_counts[value] += 1
value_counts

defaultdict(int, {2: 382, 1: 471})

In [6]:
nx.write_edgelist(new_G, 'data/yekaterinburg_small_speed_history_cleared.edgelist')